# Contextual Subspace VQE
### in the Stabilizer Subspace Projection formalism
[Contextual-Subspace VQE](https://doi.org/10.22331/q-2021-05-14-456) (CS-VQE) is predicated on a splitting of the Hamiltonian in two parts - one _noncontextual_ and one _contextual_ such that
\begin{equation}
    H = H_\mathrm{noncon} + H_\mathrm{context}.
\end{equation}
The noncontextual component gives way to an objective function that one may solve classically, whereas the contextual component is simulated on quantum hardware and provides corrections to the classical result. To ensure the quantum corrections are valid, i.e consistent with the noncontextual problem, one enforces noncontextual symmetries over the contextual Hamiltonian, in a similar procedure to [Qubit Tapering](https://arxiv.org/abs/1701.08213). 

The key difference between the two techniques is tapering symmetries are physical and therefore preserve the Hamiltonian energy spectrum exactly, whereas CS-VQE identifies pseudo-symmetries that result in a loss of information and is therefore approximate. However, one finds that in many cases CS-VQE still permits high precision simulations but at a dramatic reduction in qubit resource, even on top of qubit tapering, thus augmenting the capabilities of Noisy Intermediate-Scale Quantum (NISQ) hardware.

To see CS-VQE in action, we will go through the toy example given in [this](https://arxiv.org/pdf/2207.03451.pdf) paper. First, we define the Hamiltonian:

In [1]:
import numpy as np
from symmer.symplectic import PauliwordOp
from symmer.utils import exact_gs_energy

# define the Hamiltonian and initiate as a symmer.symplectic.PauliwordOp object
H_dict = {
    'IIYI': 0.6,'XYXI': 0.7,'XZXI': 0.7,'XZZI': 0.6,'YXYI': 0.1,'ZZZI': 0.7,'IIIZ': 0.5, 
    'XXXI': 0.1,'XXYI': 0.5,'XXZI': 0.2,'YXXI': 0.2,'YYZI': 0.2,'YZXI': 0.1,'ZYYI': 0.1
}
H = PauliwordOp.from_dictionary(H_dict)
true_gs_nrg, true_gs_vec = exact_gs_energy(H.to_sparse_matrix)

print('H ='); print(H); print()
print(f'The ground state energy of H is {true_gs_nrg}.')

H =
 0.600+0.000j IIYI +
 0.700+0.000j XYXI +
 0.700+0.000j XZXI +
 0.600+0.000j XZZI +
 0.100+0.000j YXYI +
 0.700+0.000j ZZZI +
 0.500+0.000j IIIZ +
 0.100+0.000j XXXI +
 0.500+0.000j XXYI +
 0.200+0.000j XXZI +
 0.200+0.000j YXXI +
 0.200+0.000j YYZI +
 0.100+0.000j YZXI +
 0.100+0.000j ZYYI

The ground state energy of H is -2.8192284734522626.


Next, we may initiate the `symmer.projection.CS_VQE` class, which will take our Hamiltonian as an input and construct the CS-VQE model.

In [2]:
from symmer.projection import CS_VQE

cs_vqe = CS_VQE(H)

A noncontextual/contextual partition is identied:

In [3]:
print('Noncontextual Hamiltonian:\n')
print(cs_vqe.noncontextual_operator); print()
print('Contextual Hamiltonian:\n')
print(cs_vqe.contextual_operator); print()
print('Check the noncontextual Hamiltonian is indeed noncontextual:',
      cs_vqe.noncontextual_operator.is_noncontextual)
print('The noncontextual/contextual parts sum to the original Hamiltonian?',
      cs_vqe.noncontextual_operator + cs_vqe.contextual_operator == H)

Noncontextual Hamiltonian:

 0.500+0.000j IIIZ +
 0.700+0.000j ZZZI +
 0.600+0.000j IIYI +
 0.600+0.000j XZZI +
 0.700+0.000j XZXI +
 0.700+0.000j XYXI +
 0.100+0.000j YXYI

Contextual Hamiltonian:

 0.100+0.000j ZYYI +
 0.100+0.000j YZXI +
 0.200+0.000j XXZI +
 0.200+0.000j YYZI +
 0.100+0.000j XXXI +
 0.500+0.000j XXYI +
 0.200+0.000j YXXI

Check the noncontextual Hamiltonian is indeed noncontextual: True
The noncontextual/contextual parts sum to the original Hamiltonian? True


From the noncontextual part, we identify symmetry generators - an independent set $\mathcal{S}$ such that each element $S \in \mathcal{S}$ commutes with every term of $H_\mathrm{noncon}$:

In [4]:
print('The symmetry generators are \n')
print(cs_vqe.symmetry_generators); print()
print('Check that each symmetry generator commutes with the noncontextual Hamiltonian?',
     np.all(cs_vqe.noncontextual_operator.commutes_termwise(cs_vqe.symmetry_generators)))

The symmetry generators are 

-1 IIIZ 
 1 IXYI 
-1 YIYI

Check that each symmetry generator commutes with the noncontextual Hamiltonian? True


The terms of the noncontextual Hamiltonian that are not generated by $\mathcal{S}$ form equivalence classes with respect to commutation. The noncontextual Hamiltonian may therefore be decomposed into its symmetry terms $\mathcal{G}$ and a collection of $M$ commuting cliques $\mathcal{C}_i$:
\begin{equation}
H_\mathrm{noncon} = \sum_{G \in \mathcal{G}} h_G G + \sum_{i=1}^{M} \sum_{C \in \mathcal{C}_i} h_C C
\end{equation}
Note that terms selected accross different cliques will anticommute. Finally, one may construct a clique observable by forming a linear combination over clique representatives:
\begin{equation}
A(\vec{r}) = \sum_{i=1}^M r_i C_i
\end{equation}
where $\vec{r} \in \mathbb{R}^M$ and $|\vec{r}| = 1$.


In [5]:
print('Noncontextual decomposition:\n')
for group, op in cs_vqe.decomposed.items():
    print(f'{group} terms:\n', op) 
print()
print('and the clique operator A(r) = \n')
print(cs_vqe.clique_operator)

Noncontextual decomposition:

symmetry terms:
  0.500+0.000j IIIZ
clique_0 terms:
  0.600+0.000j XZZI +
 0.700+0.000j XYXI
clique_1 terms:
  0.600+0.000j IIYI +
 0.100+0.000j YXYI
clique_2 terms:
  0.700+0.000j ZZZI +
 0.700+0.000j XZXI

and the clique operator A(r) = 

-0.658 XYXI +
 0.253 YXYI +
-0.709 XZXI


The noncontextual problem is fully paramatrized by the clique operator coefficient vector $\vec{r}$ and an assignment of $\pm1$ eigenvalues to the symmetry generators. The CS_VQE class calculates the optimal value of these at initialization with respect to the classical objective function defined in [this](https://arxiv.org/pdf/2002.05693.pdf) paper, stored as operator coefficients as seen above. We see that the noncontextual energy calculated in this way is indeed the actual ground state of the noncontextual Hamiltonian:

In [6]:
H_noncon_nrg = exact_gs_energy(cs_vqe.noncontextual_operator.to_sparse_matrix)[0]
print(f'The ground state of the noncontextual Hamiltonian is {H_noncon_nrg}')
print(f'The energy obtained from minimization of the classical objective function is {cs_vqe.noncontextual_energy}')
print(f'The error is {H_noncon_nrg - cs_vqe.noncontextual_energy}')

The ground state of the noncontextual Hamiltonian is -2.4748417658131507
The energy obtained from minimization of the classical objective function is -2.474841765812711
The error is -4.39648317751562e-13


With the noncontextual problem solved, we may turn our attention to the _contextual_ problem, from which we would like to derive quantum corrections to the noncontextual energy above. In order to do so, we choose a subset of symmetry generators to enforce over the contextual Hamiltonian, thus projecting into the corresponding stabilizer subspace (the _contextual_ subspace) and consequently performing some quantum simulation therein, e.g. Variational Quantum Eigensolver (VQE) or Quantum Phase Estimation (QPE). The difficulty is in choosing _which_ stabilizers to fix - for a small problem such as the one in question, we may look at the contextual ground state energy in each stabilizer subspace via brute force:

In [7]:
# this means enforcing all different combinations of 
print(cs_vqe.symmetry_generators, '\n')
# note the signs are the value of these terms are important
# and are defined by the noncontextual solution!


# and whether to enforce script A
print('A(r)=', cs_vqe.clique_operator)
# again coefficients are important and determined by solving the noncontextual problem

-1 IIIZ 
 1 IXYI 
-1 YIYI 

A(r)= -0.658 XYXI +
 0.253 YXYI +
-0.709 XZXI


In [8]:
from itertools import combinations

stabilizers = list(cs_vqe.symmetry_generators)


combinations_of_stabs_with_A_fixed = [(list(subset_stabs),True) for L in range(len(stabilizers)+1) for subset_stabs in combinations(stabilizers, L) ]
combinations_of_stabs_withOUT_A_fixed = [(list(subset_stabs),False) for L in range(len(stabilizers)+1) for subset_stabs in combinations(stabilizers, L) ]

# combine into one list (and sort by how many stabilizers are fixed)
combinations_brute_force = sorted([*combinations_of_stabs_with_A_fixed, *combinations_of_stabs_withOUT_A_fixed], 
                                  key=lambda x: -(len(x[0])+ int(x[1])))

# combinations_brute_force has all the different combinations of possible stabilizers
# the list contains which stabilizers to fix and the True/False flag determins whether A(r) is fixed

for fixed_stabs, scriptA_flag in combinations_brute_force:
    print('stabilizers to fix:', fixed_stabs)
    print('          fix A(r):', scriptA_flag, '\n')

stabilizers to fix: [-1.000+0.000j IIIZ,  1.000+0.000j IXYI, -1.000+0.000j YIYI]
          fix A(r): True 

stabilizers to fix: [-1.000+0.000j IIIZ,  1.000+0.000j IXYI]
          fix A(r): True 

stabilizers to fix: [-1.000+0.000j IIIZ, -1.000+0.000j YIYI]
          fix A(r): True 

stabilizers to fix: [ 1.000+0.000j IXYI, -1.000+0.000j YIYI]
          fix A(r): True 

stabilizers to fix: [-1.000+0.000j IIIZ,  1.000+0.000j IXYI, -1.000+0.000j YIYI]
          fix A(r): False 

stabilizers to fix: [-1.000+0.000j IIIZ]
          fix A(r): True 

stabilizers to fix: [ 1.000+0.000j IXYI]
          fix A(r): True 

stabilizers to fix: [-1.000+0.000j YIYI]
          fix A(r): True 

stabilizers to fix: [-1.000+0.000j IIIZ,  1.000+0.000j IXYI]
          fix A(r): False 

stabilizers to fix: [-1.000+0.000j IIIZ, -1.000+0.000j YIYI]
          fix A(r): False 

stabilizers to fix: [ 1.000+0.000j IXYI, -1.000+0.000j YIYI]
          fix A(r): False 

stabilizers to fix: []
          fix A(r): True 

In [9]:
# note the bottom case is when NO approximation is made and one returns the full problem Hamiltonian
# no contextual subspace approximation

In [10]:
from symmer.symplectic import StabilizerOp
from functools import reduce

nq_H = H.n_qubits # number of qubits for the full problem

for fixed_stabs, scriptA_flag in combinations_brute_force:
    n_stabs = len(fixed_stabs) + int(scriptA_flag)
    subspace_qubits =  nq_H - n_stabs
    message = f'*** Searching {subspace_qubits}-qubit stabilizer subspace ***'
    print('*'*len(message)); print(message);print('*'*len(message)) 
    
    if fixed_stabs:
        sym_op = reduce(lambda x,y: x+y, fixed_stabs)
        S = StabilizerOp(sym_op.symp_matrix, sym_op.coeff_vec)
    else:
        S = None
              
    print(f'Enforcing the symmetry generators\n{S}\n')
    if scriptA_flag:
        print(f'plus the clique operator\n{cs_vqe.clique_operator}\n')
        
    H_cs = cs_vqe.project_onto_subspace(stabilizers=S, enforce_clique_operator=scriptA_flag)
    H_cs_nrg = exact_gs_energy(H_cs.to_sparse_matrix)[0]
    print(f'We obtain the contextual energy error {H_cs_nrg}')
    print(f'with error {H_cs_nrg - true_gs_nrg}')
    print('qubit count of new CS-VQE H = ', H_cs.n_qubits)
    print()

*********************************************
*** Searching 0-qubit stabilizer subspace ***
*********************************************
Enforcing the symmetry generators
-1 IIIZ 
 1 IXYI 
-1 YIYI

plus the clique operator
-0.658 XYXI +
 0.253 YXYI +
-0.709 XZXI

We obtain the contextual energy error -2.474841765812711
with error 0.3443867076395515
qubit count of new CS-VQE H =  0

*********************************************
*** Searching 1-qubit stabilizer subspace ***
*********************************************
Enforcing the symmetry generators
-1 IIIZ 
 1 IXYI

plus the clique operator
-0.658 XYXI +
 0.253 YXYI +
-0.709 XZXI

We obtain the contextual energy error -2.6494886057860443
with error 0.1697398676662183
qubit count of new CS-VQE H =  1

*********************************************
*** Searching 1-qubit stabilizer subspace ***
*********************************************
Enforcing the symmetry generators
-1 IIIZ 
-1 YIYI

plus the clique operator
-0.658 XYXI +
 0.253 

/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/projection/cs_vqe.py:303: UserWarning: No stabilizers were specifed so the operator was returned
  warnings.warn('No stabilizers were specifed so the operator was returned')


In [11]:
# note the warning is just to let a user know if no stabilizers are provided then the full H is returned.

# looking at the errors obtained in each step 
# the optimal ordering of stabilizers to fix can be found

As was found in the [paper](https://arxiv.org/pdf/2207.03451.pdf) from which we took this example, it is possible to achieve high precision (in this case on the order $10^{-16}$!) in the 3-qubit contextual subspace (a saving of one qubit) stabilized by the single symmetry generator:

In [13]:
S = cs_vqe.symmetry_generators[0]

H_cs = cs_vqe.project_onto_subspace(S, enforce_clique_operator=False)
H_cs_nrg = exact_gs_energy(H_cs.to_sparse_matrix)[0]
print(f'Enforcing {S} we obtain an energy error of {H_cs_nrg - true_gs_nrg}.')
print('qubit count of reduced CS-VQE H:', H_cs.n_qubits)

Enforcing -1.000+0.000j IIIZ we obtain an energy error of -4.440892098500626e-16.
qubit count of reduced CS-VQE H: 3


Stabilizer identification is crucial to the success of CS-VQE and is not a straightforward problem to solve. In the original [CS-VQE paper](https://doi.org/10.22331/q-2021-05-14-456) a greedy search heuristic was employed, that functioned by relaxing $d$-many stabilizer conditions at a time that minimize the contextual energy. This scales as $\mathcal{O}(N^{d+1})$ where $N$ is the number of qubits in the full system. While technically scalable, it is not NISQ-friendly since each stabilizer trial requires a VQE simulation of the corresponding subspace.

In [15]:
## a user can define any linear combination of cs_vqe.symmetry_generators
## aka add different indexed terms (note do not repeat terms)! 
S = cs_vqe.symmetry_generators[1] + cs_vqe.symmetry_generators[2] # +...

## a user should define whether to enforce A(r)
enforce_A = True # True / False

H_cs = cs_vqe.project_onto_subspace(S, enforce_clique_operator=enforce_A)
H_cs_nrg = exact_gs_energy(H_cs.to_sparse_matrix)[0]
print(f'Enforcing {S}')
if enforce_A:
    print(f'and A(r)')
print(f'\n we obtain an energy error of {H_cs_nrg - true_gs_nrg}.')
print('qubit count of reduced CS-VQE H:', H_cs.n_qubits)

Enforcing  1.000+0.000j IXYI +
-1.000+0.000j YIYI
and A(r)

 we obtain an energy error of 0.3443867076395515.
qubit count of reduced CS-VQE H: 1
